In [ ]:
#| default_exp utils_api

# 🌐 HTTP Client

> Async HTTP client with automatic retry logic and auth helpers.

In [ ]:
#| export

from __future__ import annotations
import httpx
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception
from typing import Optional, Dict, Any
import logging
from nbdev.showdoc import show_doc

logger = logging.getLogger(__name__)

# Custom retry condition: only retry on 429 or 500+ status codes
def _should_retry_on_status(exception):
    """Only retry on 429 rate limit or 500+ server errors"""
    if isinstance(exception, httpx.HTTPStatusError):
        return exception.response.status_code == 429 or exception.response.status_code >= 500
    if isinstance(exception, httpx.RequestError):
        return True  # Always retry network errors
    return False

## 🎯 Overview

| Category | Functions | Purpose |
|----------|-----------|---------|
| 🔄 Client | `AsyncAPIClient` | Async HTTP with retry |
| 🔑 Auth | `bearer_token_auth`, `api_key_auth`, `oauth_token_auth` | Auth header generators |

---

## 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                    Retry Logic Flow                             │
├─────────────────────────────────────────────────────────────────┤
│  Request → Response                                             │
│     ↓                                                           │
│  Status 429 or 500+ → Retry (exponential: 2s, 4s, 8s)          │
│     ↓                                                           │
│  Status 400-499 (except 429) → Fail immediately                │
│     ↓                                                           │
│  Network error → Retry                                          │
│     ↓                                                           │
│  Max 3 attempts → Raise exception                               │
└─────────────────────────────────────────────────────────────────┘
```

## 🔄 AsyncAPIClient

| Method | Purpose |
|--------|---------|
| `AsyncAPIClient` | Async HTTP client with retry |
| `.request()` | Execute HTTP request with retry |
| `.get_json()` | GET request returning JSON |

In [ ]:
#| export

class AsyncAPIClient:
    """Async HTTP client with retry logic for external API integrations."""
    
    def __init__(
        self, 
        base_url: str,
        auth_headers: dict = None,
        timeout: int = 30
    ):
        """Initialize client with base URL, optional auth headers, and timeout."""
        self.base_url = base_url.rstrip('/')
        self.auth_headers = auth_headers or {}
        self.timeout = timeout
        self.client = None
    
    async def __aenter__(self):
        """Async context manager entry - creates httpx client."""
        self.client = httpx.AsyncClient(
            base_url=self.base_url,
            headers=self.auth_headers,
            timeout=self.timeout
        )
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Async context manager exit - closes httpx client."""
        if self.client:
            await self.client.aclose()
    
    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=2, max=10),
        retry=retry_if_exception(_should_retry_on_status),
        reraise=True
    )
    async def request(
        self,
        method: str,
        endpoint: str,
        params: dict = None,
        json: dict = None,
        headers: dict = None
    ) -> httpx.Response:
        """Execute HTTP request with automatic retry on 429/500+ errors."""
        if not self.client:
            raise RuntimeError("Client not initialized. Use 'async with' context manager.")
        
        request_headers = {**self.auth_headers, **(headers or {})}
        
        response = await self.client.request(
            method=method,
            url=endpoint,
            params=params,
            json=json,
            headers=request_headers
        )
        response.raise_for_status()
        return response
    
    async def get_json(
        self,
        endpoint: str,
        params: dict = None
    ) -> Dict[str, Any]:
        """Convenience GET that returns parsed JSON."""
        response = await self.request('GET', endpoint, params=params)
        return response.json()

In [ ]:
show_doc(AsyncAPIClient.__init__)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_api.py#L48){target="_blank" style="float:right; font-size:smaller"}

### AsyncAPIClient.__init__

>      AsyncAPIClient.__init__ (base_url:str, auth_headers:dict=None,
>                               timeout:int=30)

*Initialize self.  See help(type(self)) for accurate signature.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| base_url | str |  | The root URL for the API |
| auth_headers | dict | None | Optional dictionary of auth headers (e.g., Authorization: Bearer TOKEN) |
| timeout | int | 30 | Request timeout in seconds |

In [ ]:
show_doc(AsyncAPIClient.request)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_api.py#L79){target="_blank" style="float:right; font-size:smaller"}

### AsyncAPIClient.request

>      AsyncAPIClient.request (method:str, endpoint:str, params:dict=None,
>                              json:dict=None, headers:dict=None)

*Execute HTTP request with automatic retry on transient failures.*

Retries up to 3 times with exponential backoff (2s, 4s, 8s) on:
- 429 Too Many Requests (rate limiting)
- 500-level server errors
- Network errors (timeout, connection refused)

Does NOT retry on 400-level errors (except 429).

Args:
    method: HTTP method
    endpoint: API endpoint path
    params: Query parameters
    json: JSON request body
    headers: Additional headers

Returns:
    httpx.Response: Response object

Raises:
    httpx.HTTPStatusError: After 3 failed retries or immediately on 400-level errors
    httpx.RequestError: Network-level errors

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| method | str |  | HTTP method (GET, POST, PUT, DELETE) |
| endpoint | str |  | API endpoint path (e.g., '/users') |
| params | dict | None | Query parameters |
| json | dict | None | JSON body for POST/PUT |
| headers | dict | None | Additional headers to merge with auth_headers |
| **Returns** | **Response** |  |  |

In [ ]:
show_doc(AsyncAPIClient.get_json)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_api.py#L130){target="_blank" style="float:right; font-size:smaller"}

### AsyncAPIClient.get_json

>      AsyncAPIClient.get_json (endpoint:str, params:dict=None)

*Convenience method for GET requests that return JSON.*

Args:
    endpoint: API endpoint path
    params: Query parameters

Returns:
    Parsed JSON response as dict

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| endpoint | str |  | API endpoint path |
| params | dict | None | Query parameters |
| **Returns** | **Dict** |  |  |

## 🔑 Auth Helpers

| Function | Purpose |
|----------|---------|
| `bearer_token_auth` | Bearer token header |
| `api_key_auth` | API key header (customizable) |
| `oauth_token_auth` | OAuth 2.0 access token |

In [ ]:
#| export

def bearer_token_auth(token: str) -> dict:
    """Generate Bearer token authentication header."""
    return {'Authorization': f'Bearer {token}'}


def api_key_auth(api_key: str, header_name: str = 'X-API-Key') -> dict:
    """Generate API key header with customizable header name."""
    return {header_name: api_key}


def oauth_token_auth(access_token: str) -> dict:
    """Generate OAuth 2.0 access token header (alias for bearer_token_auth)."""
    return bearer_token_auth(access_token)

In [ ]:
show_doc(bearer_token_auth)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_api.py#L149){target="_blank" style="float:right; font-size:smaller"}

### bearer_token_auth

>      bearer_token_auth (token:str)

*Generate Bearer token authentication header.*

Args:
    token: API access token

Returns:
    Dict with Authorization header

Example:
    client = AsyncAPIClient(
        base_url='https://api.example.com',
        auth_headers=bearer_token_auth('my_token_123')
    )

In [ ]:
show_doc(api_key_auth)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_api.py#L168){target="_blank" style="float:right; font-size:smaller"}

### api_key_auth

>      api_key_auth (api_key:str, header_name:str='X-API-Key')

*Generate API key authentication header.*

Args:
    api_key: API key value
    header_name: Custom header name (default: X-API-Key)

Returns:
    Dict with API key header

Example:
    # Standard X-API-Key
    auth = api_key_auth('my_key_123')

    # Custom header name
    auth = api_key_auth('my_key_123', 'X-Custom-API-Key')

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| api_key | str |  | API key value |
| header_name | str | X-API-Key | Header name (default: X-API-Key) |
| **Returns** | **dict** |  |  |

In [ ]:
show_doc(oauth_token_auth)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_api.py#L192){target="_blank" style="float:right; font-size:smaller"}

### oauth_token_auth

>      oauth_token_auth (access_token:str)

*Generate OAuth 2.0 access token header (alias for bearer_token_auth).*

Args:
    access_token: OAuth 2.0 access token

Returns:
    Dict with Authorization header

In [ ]:
#| hide

import nbdev as nb
nb.nbdev_export()